本文件仅包含一些测试代码

In [15]:
# test config.json load and parse
import json

with open("config.json") as j:
    cfg = json.load(j)

print(cfg)

{'vast_api_key': 'a7698ab720b9525b36abe8c247cef30344d26d05cacbac80889a4b8e166fa022', 'eth_address': '0x24691e54afafe2416a8252097c9ca67557271475', 'machine_nums': 4, 'machine_configs': [{'gpu_name': 'RTX_4090', 'gpu_nums': 8, 'dph': 2}, {'gpu_name': 'RTX_3090', 'gpu_nums': 8, 'dph': 1.5}]}


In [ ]:
# test search offers
import yagmail
import time
from vastai_client import VastClient

flag = False
api_key = cfg['vast_api_key']
eth_address = cfg['eth_address']
machine_nums = cfg['machine_nums']
machine_configs = cfg['machine_configs']

def search_offers(api_key, cfg):
    global machine_nums, flag
    print(cfg)
    client = VastClient(api_key)
    available_machines = client.search_offers(
        # reliability > 0.98
        search_query=f'num_gpus = {cfg["gpu_nums"]} dph <= {cfg["dph"]} gpu_name = {cfg["gpu_name"]} '
                     'cuda_vers >= 12.0 verified=false external=false rentable=true',
        sort_order='dph, reliability-'
    )
    print(available_machines)
    if not available_machines:
        print(f"Not available {cfg['gpu_name']} machines")
        return False
    machine_nums -= 1
    # create instance
    for machine in available_machines:
        res = client.create_instance(id=machine.id, image="nvidia/cuda:12.0.1-devel-ubuntu20.04", ssh=True,
                                     env=f"ETH_ADDRESS={cfg['eth_address']}")
        print(res)
        
        # send mail
        try:
            yag = yagmail.SMTP(user='xicodez@outlook.com', host='smtp.gmail.com')
            yag.send(to='maxiaoxiong0@gmail.com', subject='Auto rend gpu result', contents=res)
            print('Email send success')
        except:
            print("send email fail:" + res)
        if machine_nums == 0:
            flag = True
            break

while not flag:
    for cfg in machine_configs:
        search_offers(api_key, cfg)
    print("sleep 5 seconds")
    time.sleep(10)

print("program done!")

{'gpu_name': 'RTX_4090', 'gpu_nums': 8, 'dph': 2}
[]
Not available RTX_4090 machines
{'gpu_name': 'RTX_3090', 'gpu_nums': 8, 'dph': 1.5}
[]
Not available RTX_3090 machines
sleep 5 seconds
{'gpu_name': 'RTX_4090', 'gpu_nums': 8, 'dph': 2}
[]
Not available RTX_4090 machines
{'gpu_name': 'RTX_3090', 'gpu_nums': 8, 'dph': 1.5}
[]
Not available RTX_3090 machines
sleep 5 seconds
{'gpu_name': 'RTX_4090', 'gpu_nums': 8, 'dph': 2}
[]
Not available RTX_4090 machines
{'gpu_name': 'RTX_3090', 'gpu_nums': 8, 'dph': 1.5}
[]
Not available RTX_3090 machines
sleep 5 seconds
{'gpu_name': 'RTX_4090', 'gpu_nums': 8, 'dph': 2}
[]
Not available RTX_4090 machines
{'gpu_name': 'RTX_3090', 'gpu_nums': 8, 'dph': 1.5}
[]
Not available RTX_3090 machines
sleep 5 seconds
{'gpu_name': 'RTX_4090', 'gpu_nums': 8, 'dph': 2}
[]
Not available RTX_4090 machines
{'gpu_name': 'RTX_3090', 'gpu_nums': 8, 'dph': 1.5}
[]
Not available RTX_3090 machines
sleep 5 seconds
{'gpu_name': 'RTX_4090', 'gpu_nums': 8, 'dph': 2}
[]
Not ava

> 记录一些原始api的测试情况

## 获取当前用户设备列表
```
url = "https://console.vast.ai/api/v0/instances?api_key=" + api_key

payload = {}
headers = {
    'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)
```

## 查询符合要求的机器
```
from urllib.parse import quote
import json

q = {
    "disk_space": {"gte": 16},
    "verified": {"eq": "true"},
    "rentable": {"eq": "true"},
    "num_gpus": {"gte": 2, "lte": 2},
    "gpu_option": {"eq": "RTX 3090"},
    "sort_option": {"0": ["dphtotal", "asc"], "1": ["total_flops", "asc"]},
    "order": [["dphtotal", "asc"]],
    "gpu_name": {"eq": "RTX 3090"},
    "allocated_storage": 16,
    "extra_ids": [],
    "type": "ask"
}
url = "https://console.vast.ai/api/v0/bundles?q=" + quote(json.dumps(q))
payload = {}
headers = {
    'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(json.dumps(json.loads(response.text), indent=4, sort_keys=True))
```

## 创建实例
```
url = "https://console.vast.ai/api/v0/asks/{id}/?api_key=" + api_key

payload = {
    "client_id": "me",
    "image": "nvidia/cuda:12.0.1-devel-ubuntu20.04",
    "runtype": "ssh ssh_direc ssh_proxy",
    "image_login": None,
    "python_utf8": False,
    "lang_utf8": False,
    "use_jupyter_lab": False,
    "jupyter_dir": None,
    "create_from": None,
    "force": False,
    "template_id": 27230
}
headers = {
    'Accept': 'application/json'
}

response = requests.request("PUT", url, headers=headers, data=payload)

print(response.text)
```

In [ ]:
# python 通过ssh远程执行命令
# import paramiko
